## 1.Import 및 라이브러리 다운로드

In [1]:
import gc
import torch
import sys

gc.collect()
torch.cuda.empty_cache()

sys.path.append('~/aiffel/Aiffelthon_koBART')

In [2]:
!pip install rouge_score
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [3]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
from datasets import Dataset



import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments

)

from transformers import RobertaTokenizerFast
from transformers import EncoderDecoderModel
from dataclasses import dataclass, field
from typing import Optional
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
from tabulate import tabulate

In [4]:
# 데이터 불러오기
train_20sent = pd.read_csv('data/train_20per_Sum3.csv')
#train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
train_20sent.rename(columns = {"input": "Text"}, inplace = True)
#train_20sent.rename(columns = {"sentence_per_20": "Summary"}, inplace = True)
train_20sent.rename(columns = {"sentence_per_20": "Summary"}, inplace = True)

val_20sent = pd.read_csv('data/val_20per_Sum3.csv')
#val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
val_20sent.rename(columns = {"input": "Text"}, inplace = True)
#val_20sent.rename(columns = {"summary_per_20": "Summary"}, inplace = True)
val_20sent.rename(columns = {"summary_per_20": "Summary"}, inplace = True)
print(len(train_20sent), len(val_20sent))

73431 9150


In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거
    sentence = re.sub('"','', sentence) # 쌍따옴표 제거
    sentence = re.sub("'",'', sentence) # 따옴표 제거
    sentence = re.sub('\n','', sentence) # \n " 제거
    sentence = re.sub('.{2,3}\W{0,1}기자','', sentence) # 기자 이름 제거
    sentence = re.sub(r'[?.!,][/?.!,]', '', sentence) # 여러개 문장 부호를 하나의 문장부호로 바꿉니다
    sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence


In [6]:
clean_text = []
clean_headlines = []

for i in tqdm(train_20sent['Text']):
    clean_text.append(preprocess_sentence(i))
for i in tqdm(train_20sent['Summary']):
    clean_headlines.append(preprocess_sentence(i))

100%|██████████| 73431/73431 [00:03<00:00, 21255.59it/s]


In [7]:
train_20sent['Text'] = clean_text
train_20sent['Summary'] = clean_headlines

In [8]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_20sent) 
val_len = len(val_20sent) // 2
val_data = Dataset.from_pandas(val_20sent[:val_len])
test_data=Dataset.from_pandas(val_20sent[val_len:])

In [9]:
# train_len = len(train_20sent)//4
# train_data = Dataset.from_pandas(train_20sent[:train_len]) 
# val_len = len(val_20sent) // 8
# val_data = Dataset.from_pandas(val_20sent[:val_len])
# test_data=Dataset.from_pandas(val_20sent[val_len:])

In [10]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 73431)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 4575)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 4575)


In [11]:
max_input = 512
max_target = 64
batch_size = 3
model_checkpoints = "gogamza/kobart-base-v2"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [13]:
def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['Text']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['Summary'], max_length=max_target, padding='max_length', truncation=True)
    
  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [14]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [16]:
# set special tokens
#from transformers import EncoderDecoderConfig
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 2
model.config.length_penalty = 2.0
model.config.num_beams = 2


In [17]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results_221109_3",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs2",
    logging_steps=500,
    save_total_limit=3,
)

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 73431
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4590
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.367100
1000,1.582800
1500,1.549700
2000,1.532600
2500,1.525600
3000,1.514400
3500,1.511200
4000,1.508800
4500,1.504500


Saving model checkpoint to results_221109_3/checkpoint-500
Configuration saved in results_221109_3/checkpoint-500/config.json
Model weights saved in results_221109_3/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results_221109_3/checkpoint-500/tokenizer_config.json
Special tokens file saved in results_221109_3/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results_221109_3/checkpoint-1000
Configuration saved in results_221109_3/checkpoint-1000/config.json
Model weights saved in results_221109_3/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results_221109_3/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results_221109_3/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results_221109_3/checkpoint-1500
Configuration saved in results_221109_3/checkpoint-1500/config.json
Model weights saved in results_221109_3/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results_221109_3/checkpo

TrainOutput(global_step=4590, training_loss=1.619514312411705, metrics={'train_runtime': 5157.9983, 'train_samples_per_second': 14.236, 'train_steps_per_second': 0.89, 'total_flos': 2.238677893251072e+16, 'train_loss': 1.619514312411705, 'epoch': 1.0})

In [22]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4575
  Batch size = 16


{'eval_loss': 1.9964174032211304,
 'eval_rouge2_precision': 0.1583,
 'eval_rouge2_recall': 0.1585,
 'eval_rouge2_fmeasure': 0.1519,
 'eval_runtime': 505.8773,
 'eval_samples_per_second': 9.044,
 'eval_steps_per_second': 0.565,
 'epoch': 1.0}

In [23]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v2")# 여기에 기본 kobart가져오기?

test_samples = val_data.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1] # 여기에 체크포인트 가져오기 
# 연구해봐야한다.

loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v2/snapshots/d9a1f640896cef8dcfd693b1bc57510a2b09a18f/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v2",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "

In [24]:
# from tabulate import tabulate

In [25]:
for i in range(len(summaries_after_tuning)):
    print('idx_{} '.format(i))
    print("Summary before \n", summaries_before_tuning[i])
    print("Summary after \n", summaries_after_tuning[i])
    print("Target summary \n", test_samples["Summary"][i])
    print('Text', test_samples["Text"][i])
    print('-'*100)
    

idx_0 
Summary before 
 일본 사법당당국의 출국금지 명령이 내려져 있던 카를로스 곤(65)
Summary after 
 일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(6 방송통신 산업통상자원 미래창조과학 한국 레바논으로 비밀리에 도주  30일(현지시간) 미 일간 월스트리트저널(WSJ)은 소식통을 인용해 곤 전 회장이 이날레바논 수도 베이루트 공항에 도착
Target summary 
 일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이 레바논으로 비밀리에 도주했다. 이 때문에 곤 전 회장의 변호인은 물론 수사·출입국 당국까지 당혹스럽다는 반응을 보이는 상황이다. 곤 전 회장은 일본의 ‘정치적 박해’로부터 빠져나왔다고 주장하고 있다.
Text 일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이 레바논으로 비밀리에 도주했다.
  30일(현지시간) 미 일간 월스트리트저널(WSJ)은 소식통을 인용해 곤 전 회장이 이날 레바논 수도 베이루트 공항에 도착했다고 전했다.
  곤 전 회장은 보도 이후 미국의 대변인을 통해 자신이 레바논에 머물고 있다는 내용의 성명을 발표했다.
  이 때문에 곤 전 회장의 변호인은 물론 수사·출입국 당국까지 당혹스럽다는 반응을 보이는 상황이다.
    곤 전 회장은 일본의 ‘정치적 박해’로부터 빠져나왔다고 주장하고 있다.
  그는 성명을 통해 “유죄가 전제되고 차별이 만연하고 기본적 인권이 무시되는 잘못된 일본 사법제도의 ‘인질’이 되지 않겠다”고 밝혔다.
  곤 전 회장은 르노닛산 회장 시절 회사 공금을 유용했다는 혐의 등으로 일본에서 재판을 받고 있다.
  지난해 4월 두 번째 보석 당시 법원에서는 거주제한·출국금지 조치를 포함한 엄격한 감시 명령을 내렸다.
  도쿄지방재판소는 도쿄도(東京都) 미나토(港)구 소재 단독 주택으로 곤의 주거지를 제한하고 일본에서 출국하는 것을 금지하는 등의 조건으로 보석을 인정했다.
  곤 전 회장의 주거지 현관에 감시

In [26]:
# print(
#     tabulate(
#         zip(
#             range(len(summaries_after_tuning)),
#             summaries_after_tuning,
#             summaries_before_tuning,
#         ),
#         headers=["Id", "Summary after", "Summary before"]
#     )
# )
# print("\nTarget summaries:\n")
# print(
#     tabulate(list(enumerate(test_samples["Summary"])), headers=["Id", "Target summary"])
# )
# print("\nSource documents:\n")
# print(tabulate(list(enumerate(test_samples["Text"])), headers=["Id", "Text"]))